In [1]:
# GA CUSTOMER REVENUE COMPETITION
# Updated kernel (11/11) with v2 files
# Read and preprocess all columns, except hits.

import gc
import os
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import json
import time
from ast import literal_eval
pd.set_option('display.max_columns', 500)
# pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# train_df = load_df('train_v2.csv',nrows=100)

def load_df(file_name = 'train_v2.csv', nrows = None,skiprows=None):
    """Read csv and convert json columns."""
    
    USE_COLUMNS = [
        'channelGrouping', 'date', 'device', 'fullVisitorId', 'geoNetwork',
        'socialEngagementType', 'totals', 'trafficSource', 'visitId',
        'visitNumber', 'visitStartTime', 'customDimensions',
        'hits'
    ]

    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    df = pd.read_csv('./data/{}'.format(file_name),
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, nrows=nrows, usecols=USE_COLUMNS,skiprows=skiprows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
        
    # Normalize customDimensions
    df['customDimensions']=df['customDimensions'].apply(literal_eval)
    df['customDimensions']=df['customDimensions'].str[0]
    df['customDimensions']=df['customDimensions'].apply(lambda x: {'index':np.NaN,'value':np.NaN} if pd.isnull(x) else x)

    column_as_df = json_normalize(df['customDimensions'])
    column_as_df.columns = [f"customDimensions_{subcolumn}" for subcolumn in column_as_df.columns]
    df = df.drop('customDimensions', axis=1).merge(column_as_df, right_index=True, left_index=True)
    
    # ===============================================
    print("---> working on hits")
    feat = 'hits'
    df[feat]=df[feat].apply(literal_eval)
    df[feat]=df[feat].str[0]
    df[feat]=df[feat].apply(lambda x: {'index':np.NaN} if pd.isnull(x) else x)
    column_as_df = json_normalize(df[feat])
    column_as_df.columns = [f"hits_{subcolumn}" for subcolumn in column_as_df.columns]
    df = df.drop(feat, axis=1).merge(column_as_df, right_index=True, left_index=True)
    
    # ===============================================
    print("---> working on hits_promotion")
    feat = 'hits_promotion'
#     df[feat]=df[feat].apply(literal_eval)
    df[feat]=df[feat].str[0]
    df[feat]=df[feat].apply(lambda x: {'index':np.NaN} if pd.isnull(x) else x)
    column_as_df = json_normalize(df[feat])
    column_as_df.columns = [f"hits_promotion_{subcolumn}" for subcolumn in column_as_df.columns]
    df = df.drop(feat, axis=1).merge(column_as_df, right_index=True, left_index=True)
    
    
    # ===============================================
    print("---> working on hits_product")
    feat = 'hits_product'
#     df[feat]=df[feat].apply(literal_eval)
    df[feat]=df[feat].str[0]
    df[feat]=df[feat].apply(lambda x: {'index':np.NaN} if pd.isnull(x) else x)
    column_as_df = json_normalize(df[feat])
    column_as_df.columns = [f"hits_product_{subcolumn}" for subcolumn in column_as_df.columns]
    df = df.drop(feat, axis=1).merge(column_as_df, right_index=True, left_index=True)
    
    bracket_col = ['hits_customDimensions','hits_customMetrics','hits_customVariables','hits_experiment',
               'hits_publisher_infos','hits_product_customDimensions','hits_product_customMetrics']
    for col in bracket_col:
        df[col] = df[col].str[0]
    
    return df

    
def pipeline():
    timer = time.time()
    %time train = load_df('train_v2.csv')
    # Drop constant columns in train and test
    const_cols = [c for c in train.columns if train[c].nunique(dropna=False) < 2]
    const_cols.append('customDimensions_index')  # Also not usefull
    train.drop(const_cols, axis=1, inplace=True)
    # Drop campaignCode (has only 1 example that is not NaN) - only on train set
    train.drop('trafficSource_campaignCode', axis=1, inplace=True)
    # Save as pickle file (could be hdf5 or feather too)
    train.to_pickle('train_all.pkl')
    print("Train shape", train.shape)
    del train; gc.collect()
    
    %time test = load_df('test_v2.csv')
    # Drop constant columns in train
    test.drop(const_cols, axis=1, inplace=True)
    # Save as pickle file (could be hdf5 or feather too)
    test.to_pickle('test_all.pkl')
    print("Test shape", test.shape)
    print("Pipeline completed in {}s".format(time.time() - timer))
    

    

In [ ]:
pipeline()
# train_df = load_df('train_v2.csv',nrows=1000,skiprows=range(1,10))
# train_df = load_df('train_v2.csv',nrows=1000)
# train_df.head(30)

---> working on hits


In [8]:
train_df['hits_promotion_promoName'].str[0]

0      NaN
1        A
2        A
3        A
4        A
5        A
6        A
7        A
8        A
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
30     NaN
31     NaN
32     NaN
33     NaN
34     NaN
35     NaN
36     NaN
37     NaN
38     NaN
39     NaN
40       A
41       A
42       A
43       A
44       A
45       A
46       A
47       A
48       A
49       A
50       A
51       A
52       A
53       A
54       A
55       A
56       A
57       A
58       A
59       A
60       A
61     NaN
62     NaN
63     NaN
64     NaN
65     NaN
66     NaN
67     NaN
68     NaN
69     NaN
70     NaN
71     NaN
72     NaN
73     NaN
74     NaN
75     NaN
76     NaN
77     NaN
78     NaN
79     NaN
80     NaN
81     NaN
82     NaN
83     NaN
84     NaN
85     NaN
86     NaN
87     NaN
88     NaN
89     NaN
90     NaN

In [18]:
print([x for x in range(1,10)])


[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [33]:
job = 1
job_per = 10

job_beg = job * job_per 
job_end = (job +1) * job_per


totoal_min = 1
total_max = 100
skip_row = [x for x in range(totoal_min,total_max) if x not in range(job_beg,job_end)]
train_df = load_df('train_v2.csv',nrows=job_per,skiprows=skip_row)
train_df.to_csv("./data/train.tmp.{}".format(job),index=False)

In [28]:
[x for x in range(1,10) if x not in range(1,3)]

[3, 4, 5, 6, 7, 8, 9]

In [34]:
train_df

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device_browser,device_browserSize,device_browserVersion,device_deviceCategory,device_flashVersion,device_isMobile,device_language,device_mobileDeviceBranding,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_mobileDeviceModel,device_mobileInputSelector,device_operatingSystem,device_operatingSystemVersion,device_screenColors,device_screenResolution,geoNetwork_city,geoNetwork_cityId,geoNetwork_continent,geoNetwork_country,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_metro,geoNetwork_networkDomain,geoNetwork_networkLocation,geoNetwork_region,geoNetwork_subContinent,totals_hits,totals_newVisits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_visits,trafficSource_adContent,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,customDimensions_index,customDimensions_value,hits_appInfo.exitScreenName,hits_appInfo.landingScreenName,hits_appInfo.screenDepth,hits_appInfo.screenName,hits_contentGroup.contentGroup1,hits_contentGroup.contentGroup2,hits_contentGroup.contentGroup3,hits_contentGroup.contentGroup4,hits_contentGroup.contentGroup5,hits_contentGroup.contentGroupUniqueViews2,hits_contentGroup.previousContentGroup1,hits_contentGroup.previousContentGroup2,hits_contentGroup.previousContentGroup3,hits_contentGroup.previousContentGroup4,hits_contentGroup.previousContentGroup5,hits_customDimensions,hits_customMetrics,hits_customVariables,hits_dataSource,hits_eCommerceAction.action_type,hits_eCommerceAction.step,hits_exceptionInfo.isFatal,hits_experiment,hits_hitNumber,hits_hour,hits_isEntrance,hits_isInteraction,hits_item.currencyCode,hits_minute,hits_page.hostname,hits_page.pagePath,hits_page.pagePathLevel1,hits_page.pagePathLevel2,hits_page.pagePathLevel3,hits_page.pagePathLevel4,hits_page.pageTitle,hits_product,hits_promotion,hits_publisher_infos,hits_referer,hits_social.hasSocialSourceReferral,hits_social.socialInteractionNetworkAction,hits_social.socialNetwork,hits_time,hits_transaction.currencyCode,hits_type
0,Organic Search,20171016,1259490915281096752,Not Socially Engaged,1508165159,2,1508165159,Safari,not available in demo dataset,not available in demo dataset,mobile,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,iOS,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Europe,Netherlands,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,Western Europe,2,NaN,2,1,32,1,NaN,not available in demo dataset,(not set),True,(not provided),organic,NaN,google,4,EMEA,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/google+redesig...,0,shop.googlemerchandisestore.com/google+redesig...,(not set),Apparel,(not set),(not set),(not set),1,(entrance),(entrance),(entrance),(entrance),(entrance),[],[],[],web,0,1,True,[],1,7,True,True,USD,45,shop.googlemerchandisestore.com,/google+redesign/apparel,/google+redesign/,/apparel,,,Apparel | Google Merchandise Store,"[{'productSKU': 'GGOEGAAX0104', 'v2ProductName...",[],[],https://www.google.nl/,No,:,(not set),0,USD,PAGE
1,Organic Search,20171016,7390444353235629134,Not Socially Engaged,1508157285,1,1508157285,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Chrome OS,not available in demo dataset,not available in dem